# Revisiting Food-Safety Inspections from the Chicago Dataset - A Tutorial

# Publication 1

## 1. Introduction

Foodborne illnesses afflict an estimated 48 million Americans each year, resulting in 128,000 hospitalizations and 3,000 fatalities [1]. City Governments curb the spread of illness by enforcing stringent food codes but often act under heavy resource constraints. Building on work by the city of Chicago, in this tutorial we seek to develop a predictive model of food code violations that could help to safeguard public health by providing real-time risk analysis both to public officials and the facilities they inspect.

### 1.1. Past Work

[Write a little bit about what we're gonna do for data processing and structuring. That we need to clean up the data and get it ready for data science.]

In 2014 the Chicago Department of Public Health (CDPH) began work on a predictive model of critical health code violations, the primary determinant of health and inspection outcomes [2][3]. After exploring a number of datasets and features, the team arrived at the following list of predictors:
* the inspector assigned
* previous inspection outcomes
* age at inspection
* licenses for alcohol consumption on premises
* license for over counter tobacco retail
* local temperature at time of inspection
* nearby burglaries
* nearby sanitation complaints
* nearby garbage cart requests.

Using a glmnet model trained on these features the team was able to demonstrate that a data-driven inspections schedule could significantly reduce the time to find a critical violation. This improved inspections schedule has since been adopted by the CDPH.

### 1.2. Premise

In light of the CDPH's success we began this project as an exploration and extension of their work. Whereas the Chicago team set out to predict critical violations, we are working to develop a series of models predicting individual violations as well as to explore neural networks for the prediction model.

By developing a higher resolution model we hope to provide actionable information not just to public officials but to restaurants themselves. In addition to being generally interested in customers' health restaurants are fined \$500 for each critical and $250 for each serious violation found, and so are highly motivated to maintain health codes. A real-time violation risk analysis could thus provide actionable information to restaurants on an ongoing basis, improving inspection outcomes and promoting public health.

## 2. Pre-processing and Data Structuring

Although we are developing a more specific model than the CDPH and using different tools to do our investigative question is similar. Thus we began by preparing those datasets that the Chicago team found to be significant, translating from R to Python and adapting code as needed.

While we may draw from additional datasets as work progresses the current list is as follows:
* [food inspections](https://data.cityofchicago.org/Health-Human-Services/Food-Inspections/4ijn-s7e5/data)
* [business licenses](https://data.cityofchicago.org/Community-Economic-Development/Business-Licenses/r5kz-chrr/data)
* [garbage cart requests](https://www.cityofchicago.org/city/en/dataset/garbage_cart_stolenormissing.html)
* [sanitation code complaints](https://data.cityofchicago.org/Service-Requests/311-Service-Requests-Sanitation-Code-Complaints/me59-5fac/data)
* [crimes](https://data.cityofchicago.org/Public-Safety/Crimes-2018/3i3m-jwuy/data)
* [weather](https://darksky.net/dev)

With the exception of weather data from darksky.net all data used is available through the [Chicago Data Portal](https://data.cityofchicago.org/). We will not be working with data on inspector assigned as this is not publicly available and would not be provide useful inisght in practice.

### 2.1. Preparation of Food Inspections Data

To download the inspections data we used Sodapy, a client for the Socrata Open Data API (the data platform used by the Chicago Data Portal):

In [1]:
%%capture
!pip install sodapy

In [2]:
import pandas as pd
import numpy as np
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofchicago.org", None)

Conveniently,  Sodapy converts all columns to snake case, removes special characters and standardizes date format by default:

In [3]:
# First 50000 results, returned as JSON from API 
# Connverted to Python list of dictionaries by sodapy.
# Column names converted to snake case, special chars removed
# Dates and location formatted
results = client.get("4ijn-s7e5", limit=50000)

# Convert to pandas DataFrame
inspections = pd.DataFrame.from_records(results)

As Socrata restricts queries to 50000 entries we then paged through the remainder to access the full dataset: 

In [4]:
# Download remaining food inspections (limit 50000 / call)
start = 50000
while results:
    results = client.get("4ijn-s7e5", limit=50000, offset=start)
    inspections = inspections.append(pd.DataFrame.from_records(results))
    start += 50000

After exploring the data and reading through the CDPH's work in R we then applied a series of filters to produce a consistent and usable dataset:

In [5]:
# Remove trailing backslash (left over from sodapy conversion of "License #" column)
inspections.rename(columns={"license_": "license"}, inplace=True)

# Drop rows with missing data
inspections.dropna(subset=["inspection_date", "license", "latitude", "longitude"], inplace=True)

# Convert latitude and longitude to float
inspections.latitude = inspections.latitude.astype(float)
inspections.longitude = inspections.longitude.astype(float)

# Drop duplicates
inspections.drop_duplicates("inspection_id", inplace=True)

# Drop "0" licenses
inspections = inspections[inspections.license != "0"]

# Only consider successful inspections
inspections = inspections[~inspections.results.isin(["Out of Business", "Business Not Located", "No Entry"])]

In addition to semi-regular canvas inspections facilities may also be inspected due to complaints or a recent failed inspection. To ensure that our data is representative of restaurants operating as usual we filtered the dataset to consist only of canvas inspections:

In [6]:
# Only consider canvas inspections (not complaints or re-inspections)
inspections = inspections[inspections.inspection_type == "Canvass"]

We also filtered inspections by facility type, to eliminate the inconsistency of data from e.g. hospitals and schools which follow different inspection schedules:

In [7]:
# Only consider restaurants and grocery stores (subject to change)
inspections = inspections[inspections.facility_type.isin(["Restaurant", "Grocery Store"])]

Finally we saved the resulting dataframe as a CSV file, ready for later use:

In [8]:
import os.path
root_path = os.path.dirname(os.getcwd())

# Save result
inspections.to_csv(os.path.join(root_path, "DATA/food_inspections_demo.csv"), index=False)

### 2.2. Preparation of Remaining Socrata Data

The process of downloading and filtering business licenses, garbage cart requests, sanitation complaints and crimes was much the same as with food inspections - for specifics please see the [CODE folder of our project repository](https://github.com/Sustainabilist/ChicagoDataAnalysis/tree/master/CODE). 

In brief:
* All datasets were formatted by sodapy and filtered to remove duplicates and missing data.
* Crimes were filtered to include only burglaries since 2010 to reduce size and ensure consistency.
* Garbage cart requests and sanitation complaints were filtered to include only completed or open requests.

### 2.3. Preparation of Weather Data

The process of pulling the weather data from the [Darksky API](https://darksky.net/dev) was straightforward and can be found in notebook 16 in the CODE folder of our project repository. To eliminate redundant API calls, the script first imports any existing weather data and checks this data first before each request.

Each weather record consists of an inspection ID, a date, the pricipitation intensity, the maxiumum temperature, the windspeed and the humidity at that date. As the inspections region only measures some twenty miles across we used a single location in central Chicago for all weather records.

### 2.4. Calculation of Violations Data
Each entry in the violations column of the food inspections dataset is made up of a number of violation/comment pairs joined into a string:

In [9]:
inspections.iloc[2].violations

'32. FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED - Comments: OBSERVED DISCOLORATION BUILDUP ON THE ICE CHUTE ATTACHED TO THE SODA FOUNTAIN. INSTRUCTED TO DETAIL CLEAN AND MAINTAIN. | 38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS REQUIRED: PLUMBING: INSTALLED AND MAINTAINED - Comments: THE FRONT EXPOSED HAND SINK IS SLOW DRAINING. INSTRUCTED TO CONTACT A PLUMBER TO PROVIDE ADEQUATE DRAINING AT ALL TIMES. | 40. REFRIGERATION AND METAL STEM THERMOMETERS PROVIDED AND CONSPICUOUS - Comments: INSTRUCTED TO PROVIDE AN INTERNAL THERMOMETER FOR THE REFRIGERATION UNIT IN THE FRONT PREPARATION AREA AND MAINTAIN.'

To make violations data more accessible we split each violations entry into a sequence of binary columns describing the presence of each violation:

In [10]:
# Split violations into binary values for each violation
def split_violations(violations):
    values_row = pd.Series([])
    
    if type(violations) == str:
        violations = violations.split(' | ')
        for violation in violations:
            index = "v_" + violation.split('.')[0]
            values_row[index] = 1
    return values_row

# 5 mins
values_data = inspections.violations.apply(split_violations)

We then generated a series of column titles of the form "v_1" for violations 1-14 (critical), 15-29 (serious) and 30-44 plus 70 (minor):

In [11]:
# Generate column names
critical_columns = [("v_" + str(num)) for num in range(1, 15)]
serious_columns = [("v_" + str(num)) for num in range(15, 30)]
minor_columns = [("v_" + str(num)) for num in range(30, 45)]
minor_columns.append("v_70")

columns = critical_columns + serious_columns + minor_columns

These column headings were then combined with the violation values and paired with the inspection id for each row:

In [12]:
# Ensure no missing columns, fill NaN
values = pd.DataFrame(values_data, columns=columns).fillna(0)

values['inspection_id'] = inspections.inspection_id

We created a separate dataframe for critical, serious and minor violation counts by pairing each inspection id with the sum of the appropriate subset of violation values:

In [13]:
# Count violations
counts = pd.DataFrame({
    "critical_count": values[critical_columns].sum(axis=1),
    "serious_count": values[serious_columns].sum(axis=1),
    "minor_count": values[minor_columns].sum(axis=1)
})

counts['inspection_id'] = inspections.inspection_id

Lastly, we saved both datasets for later use:

In [14]:
# Save violation values and counts
values.to_csv(os.path.join(root_path, "DATA/violation_values_demo.csv"), index=False)
counts.to_csv(os.path.join(root_path, "DATA/violation_counts_demo.csv"), index=False)

## 3. Exploratory Analysis
Before proceeding to the selection and calculation of features we took some time for unstructured exploration to familiarize ourselves with the data.

### 3.1. Inspections Map
In order to improve our understanading of the inspections dataset and how it relates to other location-based data we used seaborn and folium to generate an inspections heat map:

In [15]:
%%capture
!pip install folium

In [16]:
# Import necessary packages 
import folium
from folium import plugins
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [17]:
# Generate map
m = folium.Map([41.8600, -87.6298], zoom_start=10)

# Convert to (n, 2) nd-array format for heatmap
inspections_arr = inspections.sample(20000)[["latitude", "longitude"]].values

# Plot heatmap
m.add_child(plugins.HeatMap(inspections_arr.tolist(), radius=10))

### 3.2. Comments Word-cloud
To gain a sense of the feedback that facilities recieve alongside violations we concatenated all comments and used the resulting string to generate a word-cloud:

In [18]:
%%capture
!pip install folium

In [19]:
from wordcloud import WordCloud

# Extract comments from violations
def get_comments(violations):
    comments = ""
    if type(violations) == str:
        violations = violations.split(' | ')
        for violation in violations:
            violation = violation.split('Comments:')
            if len(violation) == 2:
                comments += violation[1]
    return comments

# Concatenate all comments
comments = inspections.violations.apply(get_comments).str.cat(sep=" ")

# Generate wordcloud
comments_wordcloud = WordCloud().generate(comments)

# Plot wordcloud
pylab.rcParams['figure.figsize'] = (10, 10)
plt.imshow(comments_wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

NameError: name 'pylab' is not defined

Although qualitative the results point to maintenance, cleanliness, prep areas, sinks and coolers as common areas of concern.

### 3.4. Violations Chart

For a more quantitative perspective we used the violations data to generate a simple bar graph:

In [ ]:
# List violation descriptions
descriptions = {"v_1": "Approved food sources (1)", "v_2": "Hot/cold storage facilities (2)", "v_3": "Hot/cold storage temp. (3)", "v_4": "Contaminant protection (4)", "v_5": "No sick handlers (5)", "v_6": "Proper hand washing (6)", "v_7": "Proper utensil washing (7)", "v_8": "Proper sanitizing solution (8)", "v_9": "Hot/cold water supply (9)", "v_10": "Waste water disposal (10)", "v_11": "Adequate toilet facilities (11)", "v_12": "Adequate hand washing facilities (12)", "v_13": "Control of rodents, other pests (13)", "v_14": "Correct serious violations (14)", "v_15": "No re-served food (15)", "v_16": "Protection from contamination (16)", "v_17": "Proper thawing (17)", "v_18": "Pest control, associated areas (18)", "v_19": "Proper garbage area (19)", "v_20": "Proper garbage storage (20)", "v_21": "Oversight of hazardous food (21)", "v_22": "Dishwasher maintenance (22)", "v_23": "Scrape before washing (23)", "v_24": "Proper dishwashers (24)", "v_25": "Minimize toxic materials (25)", "v_26": "Adequate customer toilets (26)", "v_27": "Supplied toilet facilities (27)", "v_28": "Visible inspection report (28)", "v_29": "Correct minor violations (29)", "v_30": "Labelled containers (30)", "v_31": "Sterile utensils (31)", "v_32": "Clean, maintain equipment (32)", "v_33": "Clean, sanitize utensils (33)", "v_34": "Clean, maintain floor (34)", "v_35": "Maintain walls & ceiling (35)", "v_36": "Proper lighting (36)", "v_37": "Toilet rooms vented (37)", "v_38": "Proper venting, plumbing (38)", "v_39": "Linen, clothing storage (39)", "v_40": "Proper thermometers (40)", "v_41": "Clean facilities, store supplies (41)", "v_42": "Ice handling, hairnets, clothes (42)", "v_43": "Ice equipment storage (43)", "v_44": "Restrict prep area traffic (44)", "v_70": "Restrict smoking (70)"}

# Rename with descriptions, sum binary values for each violation
sums = values.drop(columns=["inspection_id"]).rename(columns=descriptions).sum()

# Generate color list
colors = ["red"] * 15 + ["orange"] * 14 + ["green"] * 16

# Sort sums and colors by sum value
sum_data = pd.DataFrame({"sums": sums, "colors": colors}).sort_values("sums")

# Plot bar chart
pylab.rcParams['figure.figsize'] = (10, 10)
ax = sum_data.sums.plot(kind="barh", color=sum_data.colors)
ax.set_title("Health Code Violations", fontsize=25)
ax.set_xlabel("Violation Count", fontsize=15)
ax.invert_yaxis()
plt.show()

As this chart makes clear, the vast majority of violations are minor (v_30+), with only a scattering of serious (v_15-29) and critical (v_1-14) violations.

Using an [online chart of the Chicago health code](https://webapps1.cityofchicago.org/healthinspection/Code_Violations.jsp) we determined that the most common violations pertain to cleanliness and maintenance of floors, walls and ceilings; sanitation and maintenance of utensils and equipment; proper venting; and proper organization of food and cleaning supplies.

The most common serious violation is improper control of vermin (v_18), and the most common critical violations relate to improper storage of hot and cold foods (v_2, v_3).

### 3.5. Inspection History
The Chicago team found one of the greatest predictors of critical violations and failed inspections to be the establishment's recent inspection history. To investigate this we grouped inspections by license, shifted each group to find the previous inspection and set up a table to compare conditional likelihoods:

In [ ]:
# Sort inspections by date
inspections.sort_values(by="inspection_date", inplace=True)

# Group inspections by license and shift 1 to find previous results
inspections["previous_results"] = inspections.groupby(by="license").shift().results

# Calculate cross tabulation of results and previous results
pd.crosstab(inspections.previous_results, inspections.results)

In [ ]:
# P(fail | fail)
2416 / len(inspections[inspections["previous_results"] == "Fail"])

In [ ]:
# P(fail | pass)
4800 / len(inspections[inspections["previous_results"] == "Pass"])

We found that facilities with a previous failure were almost twice as likely to fail as those with previous passing inspections, supporting the findings of the Chicago team.

## 3.6. Economic Impact
Before investing time to develop a violations forecasting tool for restaurants and other facilities we wanted to assess whether such information would actually be valuable. For example, if businesses rarely fail inspections and generally do so in summer even perfect information would not provide much additional value. If on the other hand failed inspections are frequent and difficult to predict, such a forecast would be valuable and actionable.

To explore this we first grouped inspections by license (as all inspections for a facility share a common license). Then for each license group we determined the age, yearly inspections, failure rate and several other statistics:

In [ ]:
# Convert inspection_date to datetime format
inspections["datetime"] = pd.to_datetime(inspections["inspection_date"])

# Sort by date
inspections.sort_values("datetime", inplace=True)

# Calculate statistics for license groups
def get_stats(group):
    inspections = len(group)
    fails = len(group[group.results == 'Fail'])
    most_recent = group.iloc[-1].datetime
    days = (most_recent - group.iloc[0].datetime).days + 1
    years = days / 365.25
    return pd.Series({
        'inspections': inspections,
        'fails': fails,
        'most_recent': most_recent,
        'days': days,
        'years': years
    })

# Group by license and apply get_stats
licenses = inspections.groupby('license').apply(get_stats).reset_index()

In [ ]:
rounded_years

To begin we plotted the number licenses by age bracket, and found that after a huge number of licenses with only one inspection (years_old = 0) the die off is gradual:

In [ ]:
# Count restaurants in each rounded age bracket and plot
rounded_years = (licenses.years / 2).round() * 2

age_counts = rounded_years.value_counts().sort_index()

pylab.rcParams['figure.figsize'] = (10, 7)

fig, ax1 = plt.subplots()
index = np.arange(len(age_counts))
ax1.bar(index, age_counts)
ax1.plot(age_counts.cumsum())
ax1.set_xticks(index)
ax1.set_title("Inspection History", fontsize=20)
ax1.set_xlabel("Years Inspected", fontsize=15)
ax1.set_ylabel("Businesses", fontsize=15)

ax2 = ax1.twinx()
ax2.set_yticks([1, 2, 3, 4, 5, 5.25])
ax2.set_yticklabels(["%d%%"%(x*20) for x in range(1,6)])
plt.show()

To explore whether businesses fail enough inspections for risk analysis to be worthwhile we plotted the licenses in each yearly fails bracket:

In [ ]:
# Calculate fails per rounded year
yearly_fails = licenses.fails / np.ceil(licenses.years)

# Count restaurants in each rounded yearly fails bracket
yearly_fail_counts = yearly_fails.round(1).value_counts().sort_index()

index = np.arange(len(yearly_fail_counts))

fig, ax1 = plt.subplots()
ax1.bar(index, yearly_fail_counts)
ax1.plot(index, yearly_fail_counts.cumsum())
ax1.set_xticks(index)
ax1.set_xticklabels(yearly_fail_counts.index)
ax1.set_title("Inspections Failed Yearly", fontsize=20)
ax1.set_xlabel("Yearly Fails", fontsize=15)
ax1.set_ylabel("Businesses", fontsize=15)

ax2 = ax1.twinx()
ax2.set_yticks([1, 2, 3, 4, 5, 5.25])
ax2.set_yticklabels(["%d%%"%(x*20) for x in range(1,6)])

plt.show()

We found that after a huge number of facilities with a 100% pass rate there is a gradual die-off in yearly fails, with some 2,000 out of 15,000 businesses failing one or more yearly inspections. 

Finally we computed a cross-tabulation of failure rate and business age to explore performance and survival over time:

In [ ]:
# Chart failure rate vs. age
pd.crosstab(licenses.years_old.round(), licenses.failure_rate.round(1))

In [ ]:
# Select yearly fails series
yearly_fails = licenses.yearly_fails

# Count licenses and licenses failing 1 or more inspections yearly
print(len(licenses), len(yearly_fails[yearly_fails >= 1]))

The results suggest a high failure rate and die-off rate for businesses early on, transitioning to a lower failure rate and die-off rate for more established businesses. Early inspection failures could by a cause of closure, a symptom of adverse conditions or both.

In summary we do believe that there is sufficient opportunity for a risk analysis tool to justify development. A failed inspection costs a business at minimum $500 in fines and represents a serious risk to public health. For new and potentially struggling businesses especially an advanced assessment of health and inspection risks could prove highly valuable.

## 5. Next Steps
[UNDECIDED]
* Kernel density estimation
* Selection of features
* Calculation of model data
* Process engineering? Control charts?
* neural net for critical violations
* compare results to CDPH's model
* neural net for individual violations
* look at which things can actually be predicted and would provide actionable info
* develop forecast tool
* disseminate forecast tool

[1] https://www.cdc.gov/foodborneburden/index.html

[2] https://chicago.github.io/food-inspections-evaluation/

[3] https://github.com/Chicago/food-inspections-evaluatio]

## About the Authors
[NEEDS SOMETHING ABOUT JASON/SUSTAINABILIST?]
* David Lewis is a seasoned corporate responsibility professional working to utilize technology to help improve the health and well being of human populations through environmental stewardship. 
* Russell Hofvendahl is a web application developer with a deep-seated love of data driven decision making.

# Publication 2

## 1. In Review

## 2. Kernel Density Estimation
[HAVEN'T WRITTEN THE CODE FOR THIS]
* gaussian kde
* 90 day window
* kde for garbage, crime, sanitation
* call something other than heat map to avoid confuion?

## 3. Calculation of Model Data
We then set out derive all features originally considered by the Chicago team plus several novel features, translating from R to Python in the process. While relative weights in forecasting individual violations rather than number of critical violations may be different, we expect the significant databases and [NEEDS REPHRASING & STUFF]

[MAYBE HAVE BULLET POINTS FOR DATABASES, FEATURES]

Each row of the final training dataset represents an inspection, its outcomes, and a number of other features. [SOMETHING ABOUT TRAINING METHODOLOGY, MAYBE ABOUT FIGURING OUT WHICH FEATURES MATTER FOR WHICH]

We used the inspections database as the basis for our training data.

List of features is as follows:

### 3.1. features from food inspection data
[NEED TO WRITE THIS UP]
* used subset of inspections as basis
* merged with violation values & counts on inspection id
* created pass / fail flags
* sorted by date, shifted to get previous info
  * past fail
  * past critical, serious, minor
  * inspections with no past default to 0
* calculate time since last
  * binary first_record
  * time since last defaults to 2
  
### 3.2. features from business license data
[YET TO BE CALCULATED]
  
### 3.3. from heat maps
[YET TO BE CALCULATED]
  
### 3.4. from weather
[YET TO BE CALCULATED

## 4. Generating Model